In [4]:
import os
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain_ollama.chat_models import ChatOllama
from langchain.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever
from langchain.docstore.document import Document
from rank_bm25 import BM25Okapi
from typing import List

In [5]:


def read_pdfs(pdf_directory: str) -> List[dict]:
    """Reads all PDFs from a directory and returns a list of documents with text and metadata."""
    all_documents = []
    for filename in os.listdir(pdf_directory):
        if filename.endswith(".pdf"):
            filepath = os.path.join(pdf_directory, filename)
            try:
                loader = PyMuPDFLoader(filepath)
                documents = loader.load()
                for doc in documents:
                    # Ensure metadata has 'source'
                    if 'source' not in doc.metadata:
                        doc.metadata["source"] = filename
                    all_documents.append({"text": doc.page_content, "metadata": doc.metadata})
            except Exception as e:
                print(f"Error reading {filename}: {e}")
    return all_documents

def chunk_text(documents: List[dict], chunk_size: int = 100, chunk_overlap: int = 0) -> List[dict]:
    """Splits a list of documents into chunks with metadata."""
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    all_chunks = []
    for document in documents:
        chunks = text_splitter.split_text(document["text"])
        for chunk in chunks:
            all_chunks.append({"text": chunk, "metadata": document["metadata"]})
    return all_chunks

def store_chunks_in_chroma(chunks: List[dict], embeddings, persist_directory: str = "db"):
    """Stores text chunks with metadata in Chroma vector database."""
    texts = [chunk["text"] for chunk in chunks]
    metadatas = [chunk["metadata"] for chunk in chunks]
    vectordb = Chroma.from_texts(texts, embeddings, metadatas=metadatas, persist_directory=persist_directory)
    return vectordb

def create_bm25_index(chunks: List[dict]) -> BM25Retriever:
    """Creates a BM25 index from document chunks."""
    # Convert chunk dictionaries to Document objects
    documents = [
        Document(page_content=chunk["text"], metadata=chunk["metadata"])
        for chunk in chunks
    ]
    
    # Tokenize document page_content for BM25
    tokenized_docs = [doc.page_content.split() for doc in documents]
    bm25_index = BM25Okapi(tokenized_docs)
    
    # Wrap the BM25 index in a LangChain-compatible retriever using the correct keyword
    bm25_retriever = BM25Retriever(vectorizer=bm25_index, docs=documents)
    return bm25_retriever

# --- Main execution ---
pdf_dir = "data/docs"  # Path to your PDF directory
persist_dir = "db"     # Directory to store Chroma DB

# 1. Read PDFs and create chunks (do this once)
pdf_documents = read_pdfs(pdf_dir)
chunks = chunk_text(pdf_documents, chunk_size=500, chunk_overlap=50)

# 2. Load or create the Chroma vector store using the same chunks
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
if os.path.exists(persist_dir) and os.listdir(persist_dir):
    print("Loading existing Chroma database...")
    vectordb = Chroma(persist_directory=persist_dir, embedding_function=embeddings)
else:
    print("Chroma database not found. Processing PDFs...")
    vectordb = store_chunks_in_chroma(chunks, embeddings, persist_dir)

# 3. Create BM25 index from the already created chunks (used for keyword search)
bm25_retriever = create_bm25_index(chunks)

# 4. Initialize language model
llm = ChatOllama(
    model="llama3.2:1b",
    temperature=0.7,
    base_url="http://localhost:11434"
)

# 5. Initialize conversation memory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
    output_key='answer'
)

# 6. Create an ensemble retriever combining vector DB and BM25 retrievers
vector_retriever = vectordb.as_retriever()
ensemble_retriever = EnsembleRetriever(
    retrievers=[vector_retriever, bm25_retriever],
    weights=[0.6, 0.4]  # Adjust weights based on your use case
)

# 7. Create conversation chain with memory and ensemble retriever
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=ensemble_retriever,
    memory=memory,
    return_source_documents=True,
    chain_type="stuff"
)




c:\root\opt\workspace\langchain\cai_assignment_2\.venvcai\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\root\opt\workspace\langchain\cai_assignment_2\.venvcai\lib\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\root\opt\workspace\langchain\cai_assignment_2\.venvcai\lib\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading existing Chroma database...


C:\Users\Umesh.Pandey\AppData\Local\Temp\ipykernel_33460\3428916624.py:64: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(persist_directory=persist_dir, embedding_function=embeddings)
C:\Users\Umesh.Pandey\AppData\Local\Temp\ipykernel_33460\3428916624.py:80: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


Test question

In [4]:
# 8. Example conversation sequence
questions = [
    "What is the main topic of the document?",
    "Can you summarize the key points?",
    "What references are cited in this context?"
]

for question in questions:
    result = qa_chain.invoke({"question": question})
    
    print(f"\nQuestion: {result['question']}")
    print(f"Answer: {result['answer']}")
    
    # Safely access source documents metadata
    print("\nSources:")
    for doc in result['source_documents']:
        source = doc.metadata.get('source', 'Unknown source')
        page = doc.metadata.get('page', 'N/A')
        print(f"- {source} (page {page})")


Question: What is the main topic of the document?
Answer: The main topic of the document appears to be "Adapting existing practices for handling datasets and models in machine learning (ML)" specifically in the context of COVID-19 pandemic, as it mentions projects like Data Version Control (DVC) and model cards. However, the broader theme seems to be about developing approaches to handle datasets and models in a coherent and reliable way across different types of artifacts.

Sources:
- data/docs\Challenges in Deploying Machine Learning - a Survey of Case Studies.pdf (page 25)
- data/docs\Challenges in Deploying Machine Learning - a Survey of Case Studies.pdf (page 19)
- data/docs\Challenges in Deploying Machine Learning - a Survey of Case Studies.pdf (page 1)
- data/docs\Challenges in Deploying Machine Learning - a Survey of Case Studies.pdf (page 19)
- data/docs\Challenges in Deploying Machine Learning - a Survey of Case Studies.pdf (page 11)
- data/docs\Challenges in Deploying Machi

CLI interface for user input

In [7]:
while True:
    user_input = input("\nYour question (type 'exit' to quit): ")
    if user_input.lower() in ['exit', 'quit']:
        break
    result = qa_chain.invoke({"question": user_input})
    print(f"\nQuestion: {result['question']}")
    print(f"\nANSWER: {result['answer']}")
    
    if result['source_documents']:
        print("\nReference Documents:")
        for doc in result['source_documents'][:3]: 
            source = doc.metadata.get('source', 'Unknown source')
            page = doc.metadata.get('page', 'N/A')
            print(f"- {source} (page {page})")


Question: what is mlops

ANSWER: ML-Ops (Machine Learning Operations) refers to the practices and procedures involved in building, deploying, managing, and maintaining machine learning models in production environments. This includes activities such as data collection, model training, deployment, monitoring, and troubleshooting.

Reference Documents:
- data/docs\Challenges in Deploying Machine Learning - a Survey of Case Studies.pdf (page 10)
- data/docs\Challenges in Deploying Machine Learning - a Survey of Case Studies.pdf (page 10)
- data/docs\Challenges in Deploying Machine Learning - a Survey of Case Studies.pdf (page 21)
